In [90]:
import pandas
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.metrics import accuracy_score
from datetime import datetime

In [91]:
# Read Data
csv_file = "train.csv"
csv_data = pandas.read_csv(csv_file)
test_file = "test.csv"
test_csv = pandas.read_csv(test_file)
tripIds = test_csv["tripid"]
csv_data.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [92]:
# Modify target label
csv_data.replace(to_replace='correct', value=1, inplace=True)
csv_data.replace(to_replace='incorrect', value=0, inplace=True)

X = csv_data.iloc[:, 1:-1]
Y = csv_data['label']

In [93]:
# Handling missing values
X['additional_fare'].fillna(10.5, inplace=True)
X['meter_waiting'].fillna(X['meter_waiting'].mean(), inplace=True)
X['meter_waiting_fare'].fillna(X['meter_waiting_fare'].mean(), inplace=True)
X['meter_waiting_till_pickup'].fillna(X['meter_waiting_till_pickup'].mean(), inplace=True)
X['fare'].fillna(X['additional_fare']+X['meter_waiting_fare'], inplace=True)
X['duration'].fillna(0, inplace = True)

In [94]:
# Drop set of columns
X.drop(['pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon'], axis=1, inplace=True)
test_csv.drop(['tripid','pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon'], axis=1, inplace=True)

In [95]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 100)

In [96]:
# PCA
pca = PCA(6)
pca.fit(X_train)

X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [97]:
model = xgb.XGBClassifier()
model.fit(X_train, Y_train)
Y_predict = model.predict(X_test)
accuracy_score(Y_test, Y_predict)

0.9479045401629802

In [98]:
pca = PCA(6)
pca.fit(X)

X = pca.transform(X)
T = pca.transform(test_csv)

model =xgb.XGBClassifier()
model.fit(X, Y)
T_Predict = model.predict(T)

T_Predict

array([1, 0, 1, ..., 1, 1, 1])

In [99]:
final = pandas.DataFrame()
final["tripid"] = tripIds
final["prediction"] = T_Predict
final.to_csv('final'+ datetime.now().strftime('%m-%d-%H_%M_%S'), header=True, index=False, line_terminator='\n')